In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

** Problem 1 **: (10 pts) In many cases, getting convergence rates via pen and paper is tricky.  However, we can often readily find them via numerical simulation.  

a.) (4 pts) For 
$$
f(x) = x^{2}\cos^{3}(x) - \frac{1}{2}\sin(2x), 
$$
find all the roots on the interval $[0,5]$ using the Secant method with a tolerance of $10^{-6}$.


b.) (6 pts) For each root, find the rate of convergence.  Please provide a well-labeled and well-explained figure describing the rate of convergence for each fixed point.  Does your rate change based on your initial guess?  Again, please elaborate and show plots to make your case.

** Problem 2 ** : (10 pts) So, Newton's and the Secant Method both fail if at the root $x_{\ast}$, $f'(x_{\ast})=0$.  Thus, we should not really use these methods, or at least we should stop using them if we get too close to a root which is also a critical point of a function.  However, the Bisection Method works regardless of whether a root is a critical point or not. 

Thus, develop an algorithm which blends the Bisection and Secant Method.  It should find a root to a specified tolerance, say `tol`.  You also must include a second tolerance, say `tlsw`, such that, for a given iteration of your method, if 

$$
\left|\frac{f(x_{n})-f(x_{n-1})}{x_{n}-x_{n-1}}\right| > tlsw,
$$

then you use the Secant Method, otherwise, you use the Bisection Method.  Using your method, find all the roots of 

$$
f(x) = x^{4} - 2(\pi - 1)x^3 + (1-4\pi+4\pi^{2})x^2 + 2\pi(\pi-1)x + \pi^{2}.
$$

Determine a rate of convergence for your method for different values of `tol` and `tlsw`.  How does changing these parameters influence your rate of convergence?